# 교보문고 웹크롤링

In [1]:
from bs4 import BeautifulSoup 
import requests 
import urllib.request
from selenium.webdriver.common.keys import Keys 
from urllib.request import urlretrieve
import time 
import re 
import pandas as pd
import numpy as np
from selenium import webdriver
from multiprocessing import Pool
import multiprocessing
import concurrent.futures

## 링크 가져오기

In [6]:
def getPageLinksWantRange(startPageNo, lastPageNo): 
    
    links = [] 
    path='C:/Users/qkrdb/Downloads/chromedriver.exe'
    chrome = webdriver.Chrome(path)    
    
    for pageN in range(startPageNo-1, lastPageNo):
       
        chrome.get("http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?range=1&kind=2&orderClick=DAB&mallGb=KOR&linkClass=A")
        chrome.implicitly_wait(3)
        if pageN!=0:
            pathpage = '//*[@id="main_contents"]/div[4]/div[1]/ul/li[' + str(pageN + 1) + ']/a'
            chrome.find_element_by_xpath(pathpage).click()
        
        for num in range(1, 21): 
            
            pathbook = '//*[@id="main_contents"]/ul/li[' + str(num) + ']/div[2]/div[2]/a'
            link =chrome.find_element_by_xpath(pathbook).get_attribute('href')
            links.append(link)
        
    return links

In [8]:
koybo_links = getPageLinksWantRange(1, 10)

In [9]:
koybo_links

['http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9788901243665',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791164135592',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791188331796',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791190382175',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791165382674',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9788965422785',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791130629636',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9788932920337',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791196831080',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791187481720',


## 책 정보, 리뷰 가져오기

In [15]:
def getinfo(links):

    path='C:/Users/qkrdb/Downloads/chromedriver.exe'
    chrome = webdriver.Chrome(path)
    
    book_title = "div.box_detail_point h1.title strong" 
    book_author = "div#container div.box_detail_point div.author span.name"
    book_topic = "ul.list_detail_category a"
    book_isbn = "table.table_simple2.table_opened.margin_top10 span" 
    book_review = "div.box_detail_result ul.list_detail_booklog div.content p"
    book_info1 = "div#wrap h3.title_detail_basic2"
    book_info2 = 'div#wrap div.box_detail_article'

    global book_list
    book_list=pd.DataFrame()

    for link in links:
        try:
            chrome.get(link)
            # 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다. implicitly.wait()
            chrome.implicitly_wait(3)
            DB = pd.DataFrame()
        
            title = chrome.find_elements_by_css_selector(book_title)[0].text
            author = chrome.find_elements_by_css_selector(book_author)[0].text
            author = author + ' 저'
            topic = chrome.find_elements_by_css_selector(book_topic)[0].text
            isbn = chrome.find_elements_by_css_selector(book_isbn)[0].text
            info1 = chrome.find_elements_by_css_selector(book_info1)[0].text
            info2 = chrome.find_elements_by_css_selector(book_info2)[0].text
            info = info1+info2
            info = info.replace('\n', ' ')

            #문자열 나누기(split), 양쪽 공백 지우기(strip)
            title = title.split("(")[0].strip()
         
            DB['title']=[title]
            DB['author']=[author]
            DB['topic']=[topic]
            DB['isbn']=[isbn]
            DB['info']=[info]
        
            window_before = chrome.window_handles[0]
            try:
                chrome.find_element_by_xpath('//*[@id="container"]/div[5]/div[1]/div[4]/h2[2]/small/a[1]').click()
            except:
                try:
                    chrome.find_element_by_xpath('//*[@id="container"]/div[5]/div[1]/div[3]/h2[2]/small/a[1]').click()
                except:
                    pass
        
            chrome.implicitly_wait(3)
            window_after = chrome.window_handles[1]
            # 새 창으로 이동
            chrome.switch_to_window(window_after)
        
            reviews=''
       
            try:
                for i in range(1, 11):
                    if i!=1:
                        nextpage = '/html/body/div/div[2]/ul/li[' + str(i) + ']/a'
                        nextpage = chrome.find_element_by_xpath(nextpage).click()
                        r = chrome.find_elements_by_css_selector(book_review) 
                    else:
                        r = chrome.find_elements_by_css_selector(book_review) 
              
                    for j in range(len(r)):
                        reviews += r[j].text
            except:
                pass
        
            chrome.close()   
            chrome.switch_to_window(window_before)
        
            
            DB['review'] = reviews.replace('\n','')
            book_list = book_list.append(DB,ignore_index=True)
        except:
            pass
        
#book_list = book_list.reset_index(drop=True)

In [16]:
start = time.time()
getinfo(koybo_links[0:199])
print("time :", time.time() - start)

C:\Users\qkrdb\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: use driver.switch_to.window instead
C:\Users\qkrdb\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: DeprecationWarning: use driver.switch_to.window instead


time : 4184.069779634476


In [17]:
book_list.head(200)

,title,author,topic,isbn,info,review
0,김미경의 리부트,김미경 저,자기계발,9788901243665,코로나 이후 낯선 세상에서 ‘나는 어떻게 살 것인가’ 달라진 세상에서 개인의 일과 ...,"p29 본문 중 최재천 교수님의 말""저는 사회적 거리두기를 '행동 백신'이라고 불러..."
1,돈의 속성,김승호 저,경제/경영,9791188331796,최상위 부자 김승호 회장이 직접 밝히는 돈에 통찰과 철학 맨손에서 종잣돈을 만들고 ...,어떤 책이 좋은지 알 수가 없었다. 누군가는 이런 책을 권하고 누구는 다른 면의 책...
2,더 해빙,"이서윤 , 홍주연 저",자기계발,9791190382175,"마음가짐을 다듬으며 원하는 것을 얻는 놀라운 가르침!부와 행운을 만나는 출발점, 마...",베스트셀러 순위 상위권을 오래 차지하고 있기도 하고너무 대단한 이력을 가지고 있는 ...
3,설민석의 한국사 대모험. 14,"설민석 , 스토리박스 저",어린이(초등),9791165382674,X맨의 독약을 먹고 쓰러진 로빈! 로빈을 살리기 위한 설쌤 일행의 대모험이 시작됩니...,아이들이 좋아하는 설민석의 '한국사 대모험' 시리즈. 이번에는 오랜만에 신간이 나왔...
4,애쓰지 않고 편안하게,김수현 저,시/에세이,9791130629636,“어떤 순간에도 만만하지 않은 평화주의자가 될 것!” 《나는 나로 살기로 했다》 김...,'애쓰지 않고 편안하게' 제목만으로 위로가 되고 힘이 되는 느낌을 받았습니다. 인간...
5,기억. 1,베르나르 베르베르 저,소설,9788932920337,기억의 문이 열리고 모험이 시작된다!한국 독자들이 사랑하는 베스트셀러 작가 베르나르...,"“당신이라고 믿는 게 당신의 전부가 아닙니다. 당신은 누구인가요, 당신이 진정 누구..."
6,부의 대이동,오건영 저,경제/경영,9791196831080,"모두가 주식과 부동산으로 몰려가는 이때,부자들은 왜 달러와 금에 주목하는가? 「경제...",서점 지나가다가 베스트셀러 1위책이기도 하고 왠지 책이 좋아보여서 평소 재테크에 관...
7,존리의 부자되기 습관,존 리 저,경제/경영,9791187481720,돈을 위해 일하지 말고 돈이 당신을 위해 일하게 하라!! 열심히 일하는데 왜 돈에 ...,한국 사회에 만연한 그릇된 소비행태는 한국 사람들이 돈에 대해 잘못 알고 있는 탓에...
8,성숙한 어른이 갖춰야 할 좋은 심리 습관,류쉬안 저,자기계발,9791190456098,"어제보다 확실하게 더 나은 내가 되는 법, 그 33가지 심리 습관!생각을 바꾸고 세...",이 책은 성숙한 어른이 갖춰야 할 33가지 심리 습관을 part1에서는 유리멘탈과 ...
9,나를 숨 쉬게 하는,김이나 저,시/에세이,9791190786355,대한민국 대표 작사가 김이나가 일상의 언어들에서 포착한 마음의 풍경 매 순간 결핍과...,삶에서 맞부딪히는 복잡한 감정과 관계의 고민에 대한 해법을 일상의 단어 속에서 탐색...


### index 수정

In [ ]:
#book_list = book_list.reset_index(drop=True)

### topic 삭제 및 변경

In [18]:
#소설 #국내소설 #국외소설 #시/에세이
#어린이(초등) #가정생활 #인문 #정치사회 #경제경영 
#건강 #교양과학 #외국어 #예술 #취미/스포츠
#TOEIC/TOEFL #유아(0~7세) #종교 #아동만화 #요리/와인 
#역사/문화 #자기계발 #여행 #기술/컴퓨터 #만화 #장르소설

bookinfo = book_list.copy()

bookinfo['index1'] = bookinfo['topic'].isin(['어린이(초등)', '외국어', '취미/스포츠', 
                                             'TOEIC/TOEFL', '유아(0~7세)', '아동만화', '만화', '장르소설', '요리'])
indexNames = bookinfo[ bookinfo['index1'] == True].index
bookinfo.drop(indexNames, inplace=True)
bookinfo = bookinfo.drop('index1', axis=1).reset_index(drop=True)
bookinfo
# 열의 값 변경 bookinfo.loc[bookinfo['topic'] == "어린이", 'topic'] = '어린이들'

,title,author,topic,isbn,info,review
0,김미경의 리부트,김미경 저,자기계발,9788901243665,코로나 이후 낯선 세상에서 ‘나는 어떻게 살 것인가’ 달라진 세상에서 개인의 일과 ...,"p29 본문 중 최재천 교수님의 말""저는 사회적 거리두기를 '행동 백신'이라고 불러..."
1,돈의 속성,김승호 저,경제/경영,9791188331796,최상위 부자 김승호 회장이 직접 밝히는 돈에 통찰과 철학 맨손에서 종잣돈을 만들고 ...,어떤 책이 좋은지 알 수가 없었다. 누군가는 이런 책을 권하고 누구는 다른 면의 책...
2,더 해빙,"이서윤 , 홍주연 저",자기계발,9791190382175,"마음가짐을 다듬으며 원하는 것을 얻는 놀라운 가르침!부와 행운을 만나는 출발점, 마...",베스트셀러 순위 상위권을 오래 차지하고 있기도 하고너무 대단한 이력을 가지고 있는 ...
3,애쓰지 않고 편안하게,김수현 저,시/에세이,9791130629636,“어떤 순간에도 만만하지 않은 평화주의자가 될 것!” 《나는 나로 살기로 했다》 김...,'애쓰지 않고 편안하게' 제목만으로 위로가 되고 힘이 되는 느낌을 받았습니다. 인간...
4,기억. 1,베르나르 베르베르 저,소설,9788932920337,기억의 문이 열리고 모험이 시작된다!한국 독자들이 사랑하는 베스트셀러 작가 베르나르...,"“당신이라고 믿는 게 당신의 전부가 아닙니다. 당신은 누구인가요, 당신이 진정 누구..."
5,부의 대이동,오건영 저,경제/경영,9791196831080,"모두가 주식과 부동산으로 몰려가는 이때,부자들은 왜 달러와 금에 주목하는가? 「경제...",서점 지나가다가 베스트셀러 1위책이기도 하고 왠지 책이 좋아보여서 평소 재테크에 관...
6,존리의 부자되기 습관,존 리 저,경제/경영,9791187481720,돈을 위해 일하지 말고 돈이 당신을 위해 일하게 하라!! 열심히 일하는데 왜 돈에 ...,한국 사회에 만연한 그릇된 소비행태는 한국 사람들이 돈에 대해 잘못 알고 있는 탓에...
7,성숙한 어른이 갖춰야 할 좋은 심리 습관,류쉬안 저,자기계발,9791190456098,"어제보다 확실하게 더 나은 내가 되는 법, 그 33가지 심리 습관!생각을 바꾸고 세...",이 책은 성숙한 어른이 갖춰야 할 33가지 심리 습관을 part1에서는 유리멘탈과 ...
8,나를 숨 쉬게 하는,김이나 저,시/에세이,9791190786355,대한민국 대표 작사가 김이나가 일상의 언어들에서 포착한 마음의 풍경 매 순간 결핍과...,삶에서 맞부딪히는 복잡한 감정과 관계의 고민에 대한 해법을 일상의 단어 속에서 탐색...
9,코로나 이후의 세계,제이슨 솅커 저,경제/경영,9791158740757,누구도 경험하지 못한 미래를 선점하라! 블룸버그 선정 세계 1위 미래학자 제이슨 솅...,요즘 잠잠하나 싶었던 코로나가 다시금 활개를 펼치고 있다. 코로나 종식을 선언했던 ...


## 책 이미지 가져오기

In [19]:
def getimg(isbnList):
    
    try:
        path='C:/Users/qkrdb/Downloads/chromedriver.exe'
        div = webdriver.Chrome(path)
        for isbn in isbnList:
            url = 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode='+ isbn
            div.get(url)
            pathimg = "C:/Users/qkrdb/졸작/"
            div.implicitly_wait(3)
            src = div.find_element_by_xpath('//*[@id="container"]/div[2]/form/div[2]/div[1]/div/a/img').get_attribute('src')
            #print(src)
            #print(pathimg + isbn + ".jpg")
            urlretrieve(src, pathimg + isbn + ".jpg")

    except:
         print(isbn + ' 저장 실패')
    
    div.close()

In [20]:
isbnList = bookinfo['isbn'].tolist()
start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.submit(getimg, isbnList[:50])
    executor.submit(getimg, isbnList[51:100])
    executor.submit(getimg, isbnList[101:150])
    executor.submit(getimg, isbnList[151:])
print("time :", time.time() - start)

time : 163.70098328590393


## .csv 파일로 내보내기

In [21]:
bookinfo.to_csv("kyobo.csv", encoding='utf-8-sig')
# 1개 행에 문제 있음